# Google ColabでLPM探索を実行 (Atariのみ)

このノートブックは、Craftiumのインストールをスキップし、Atari環境で `train_lpm.py` スクリプトを実行します。

## 1. セットアップ
Atari環境と録画に必要なシステム依存関係をインストールします。

In [7]:
# GPUの確認
!nvidia-smi

Sun Jan 11 10:43:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   32C    P8             16W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
!apt-get update
!apt-get install -y ffmpeg xvfb libsdl2-dev python3-opengl cmake zlib1g-dev

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,600 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,968 kB]
Fetched 5,956 kB in 3s

## 2. Pythonライブラリのインストール
AtariをサポートするGymnasiumとその他のユーティリティをインストール

In [9]:
!pip install gymnasium[classic_control,atari,accept-rom-license] \
    moviepy imageio wandb opencv-python matplotlib tqdm pillow "gymnasium>=1.0.0" shimmy

## 3. Explorationモジュールの作成
train_lpm.pyの実行に必要な `exploration` モジュールのファイルを作成。

In [10]:
from google.colab import drive
drive.mount('/content/drive')

import os
# CHANGE THIS PATH to where you uploaded your folder
# e.g., '/content/drive/MyDrive/WorldModel/LPM_exploration'
try:
    os.chdir('/content/drive/MyDrive/world_model/最終課題')
    print(f"カレントディレクトリ: {os.getcwd()}")
except Exception as e:
    print(f"ドライブのマウントまたはパスの検索に失敗しました: {e}")
    print("ファイルがアップロードされているか、パスが正しいか確認してください。")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
カレントディレクトリ: /content/drive/MyDrive/world_model/最終課題


In [ ]:
# import os

# # explorationディレクトリの作成
# os.makedirs("exploration", exist_ok=True)

# # __init__.pyの作成
# with open("exploration/__init__.py", "w") as f:
#     f.write("")

# print("explorationディレクトリと__init__.pyを作成しました")

explorationディレクトリと__init__.pyを作成しました


In [ ]:
# %%writefile exploration/cifar.py
# import math
# import random
# import numpy as np
# import torch
# import torchvision
# import torchvision.transforms as transforms
# from PIL import Image

# def load_cifar_dataset():
#     """
#     Load the CIFAR-10 dataset.
#     Returns:
#         cifar_dataset: The loaded CIFAR-10 dataset
#     """
#     # Define transform
#     transform = transforms.Compose([transforms.ToTensor()])
#     # Download dataset if not already downloaded
#     cifar_dataset = torchvision.datasets.CIFAR10(
#         root='./data',
#         train=True,
#         download=True,
#         transform=transform
#     )
#     return cifar_dataset

# def generate_random_cifar_observation(cifar_dataset, width=32, height=32):
#     """
#     Generate a random observation from CIFAR-10 dataset with the specified dimensions.
#     The function stretches the image to fill the observation space.
#     Args:
#         cifar_dataset: The CIFAR-10 dataset
#         width: Width of the observation
#         height: Height of the observation
#     Returns:
#         observation: numpy array with shape (height, width, 3)
#     """
#     # Select a random image
#     random_index = random.randint(0, len(cifar_dataset) - 1)
#     image, _ = cifar_dataset[random_index]

#     # Convert tensor to PIL Image
#     image_np = image.permute(1, 2, 0).numpy()  # Change from CxHxW to HxWxC
#     pil_image = Image.fromarray((image_np * 255).astype(np.uint8))

#     # Stretch the image to the desired dimensions
#     stretched_image = pil_image.resize((width, height), Image.Resampling.BILINEAR)

#     # Convert to numpy array format
#     observation = np.array(stretched_image)
#     return observation

# def create_cifar_function_simple():
#     """
#     Create a simple get_random_cifar function using your existing code

#     Returns:
#         Function that returns random CIFAR images (32, 32, 3) uint8
#     """
#     print("Loading CIFAR-10 dataset...")
#     cifar_dataset = load_cifar_dataset()
#     print(f"✅ Loaded {len(cifar_dataset)} CIFAR-10 images")

#     def get_random_cifar():
#         """Return a random CIFAR image as numpy array (32, 32, 3) uint8"""
#         return generate_random_cifar_observation(cifar_dataset, width=32, height=32)

#     return get_random_cifar

# # Simple usage for NoisyTV wrapper
# def create_cifar_function_for_noisy_tv(dataset_type='cifar10'):
#     """
#     Create CIFAR function for NoisyTV wrapper (simplified version)

#     Args:
#         dataset_type: 'cifar10' only (using your existing code)

#     Returns:
#         Function that returns random CIFAR images
#     """
#     if dataset_type.lower() != 'cifar10':
#         print(f"Warning: Only cifar10 supported with simple loader, got {dataset_type}")

#     return create_cifar_function_simple()


Writing exploration/cifar.py


In [ ]:
# %%writefile exploration/noisy_wrapper.py
# """
# Debug CIFAR Wrapper - Adds extensive logging to find the issue
# """

# import numpy as np
# import random
# import matplotlib.pyplot as plt
# import gymnasium as gym
# from gymnasium import spaces

# class NoisyTVEnvWrapperCIFAR(gym.Wrapper):
#     def __init__(self, env, get_random_cifar, num_random_actions=1):
#         super().__init__(env)

#         self.get_random_cifar = get_random_cifar
#         self.num_random_actions = num_random_actions

#         # Store original action space info
#         self.original_actions = self.env.action_space.n
#         self.random_actions = list(range(self.original_actions, self.original_actions + num_random_actions))

#         # Update action space
#         self.action_space = spaces.Discrete(self.original_actions + num_random_actions)

#         # Test CIFAR function
#         try:
#             test_cifar = self.get_random_cifar()
#         except Exception as e:
#             print(f"   ❌ CIFAR test failed: {e}")

#     def step(self, action):

#         if action in self.random_actions:

#             # Execute NOOP instead
#             obs, reward, terminated, truncated, info = self.env.step(0)

#             # Apply CIFAR replacement
#             obs_replaced = self.add_noisy_tv(obs)

#             return obs_replaced, reward, terminated, truncated, info
#         else:
#             return self.env.step(action)

#     def add_noisy_tv(self, obs):
#         obs = obs.copy()
#         """Replace observation with CIFAR-based noise"""
#         try:
#             # Get random CIFAR image
#             cifar_img = self.get_random_cifar()  # Should be (32, 32, 3)

#             # Convert to grayscale
#             if len(cifar_img.shape) == 3 and cifar_img.shape[2] == 3:
#                 cifar_gray = np.dot(cifar_img, [0.2989, 0.5870, 0.1140]).astype(np.uint8)
#             else:
#                 cifar_gray = cifar_img

#             # Create replacement matching obs shape exactly
#             replacement = self._create_replacement(cifar_gray, obs)

#             return replacement

#         except Exception as e:
#             import traceback
#             traceback.print_exc()

#             # Fallback: simple random noise
#             replacement = np.random.randint(0, 256, size=obs.shape, dtype=obs.dtype)
#             return replacement

#     def _create_replacement(self, cifar_gray, target_obs):
#         """Create replacement that exactly matches target observation"""
#         target_shape = target_obs.shape
#         target_dtype = target_obs.dtype

#         if len(target_shape) == 3:
#             # 3D observation (H, W, C)
#             h, w, c = target_shape

#             # Tile CIFAR to cover the area
#             tile_h = (h + 31) // 32
#             tile_w = (w + 31) // 32

#             tiled = np.tile(cifar_gray, (tile_h, tile_w))
#             cropped = tiled[:h, :w]

#             if c == 1:
#                 replacement = cropped.reshape(h, w, 1)
#             elif c == 3:
#                 replacement = np.stack([cropped] * 3, axis=2)
#             else:
#                 replacement = np.stack([cropped] * c, axis=2)

#         elif len(target_shape) == 2:
#             # 2D observation (H, W)
#             h, w = target_shape
#             tile_h = (h + 31) // 32
#             tile_w = (w + 31) // 32

#             tiled = np.tile(cifar_gray, (tile_h, tile_w))
#             replacement = tiled[:h, :w]

#         else:
#             replacement = np.random.randint(0, 256, size=target_shape, dtype=target_dtype)

#         # Ensure correct dtype
#         replacement = replacement.astype(target_dtype)

#         return replacement

#     def get_action_meanings(self):
#         """Get action meanings including CIFAR actions"""
#         if hasattr(self.env, 'get_action_meanings'):
#             meanings = self.env.get_action_meanings()
#         else:
#             meanings = ['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'DOWN']

#         meanings = list(meanings)
#         for i in range(self.num_random_actions):
#             meanings.append(f'CIFAR_DEBUG_{i+1}')

#         return meanings


Writing exploration/noisy_wrapper.py


## 5. WandB ログイン
実験記録のためにWandBにログイン \
事前にAPIキーを取得しておいてください． \
実行すると最初に3つの選択肢が出てきますが2の"use an exitstin W&B account"選択して，その後でAPIキーの入力を求められます． \
記録の必要がなければ3を選択すると記録せずに実行します．この場合はAPIキーを求められません．

In [11]:
import wandb
wandb.login()

True

## 6. 学習の実行 (Atari)
Atari Breakoutの学習スクリプトを実行します。 \
実行コマンド引数
- --env-name  実行するAtari環境の名前を入れて実行します．デフォルトはボール崩しになってます．
- --steps  学習を行うステップ数です．colab上でボール崩しだと1分で10000ステップぐらい学習しました．
- wandb  WandBで記録するようにしています．

In [12]:
import os
os.environ['MUJOCO_GL'] = 'egl'

# Run training for Atari Breakout
!python train_lpm_2.py --env-name "ALE/Breakout-v5" --steps 1000000 --wandb --noisy-tv

ストリーミング出力は最後の 5000 行に切り捨てられました。
Training Steps:   3% 32487/1000000 [10:57<9:06:01, 29.53step/s, ext_r=0.00, wm=52.02, epD=21681.08, intrD=0.07]Episode 159 ExtReward: 0.0055
Training Steps:   3% 32766/1000000 [11:02<4:33:19, 58.98step/s, ext_r=0.00, wm=55.35, epD=22066.25, intrD=0.10]Episode 160 ExtReward: 0.0072
Training Steps:   3% 32933/1000000 [11:05<4:16:25, 62.85step/s, ext_r=0.00, wm=52.00, epD=33848.37, intrD=0.10]Episode 161 ExtReward: 0.0060
Training Steps:   3% 33096/1000000 [11:09<4:13:13, 63.64step/s, ext_r=0.00, wm=49.25, epD=28732.64, intrD=0.10]Episode 162 ExtReward: 0.0062
Training Steps:   3% 33347/1000000 [11:14<3:56:06, 68.24step/s, ext_r=0.00, wm=51.66, epD=51348.17, intrD=0.12]Episode 163 ExtReward: 0.0081
Training Steps:   3% 33512/1000000 [11:17<3:54:20, 68.74step/s, ext_r=0.00, wm=49.72, epD=35512.76, intrD=0.11]Episode 164 ExtReward: 0.0060
Training Steps:   3% 33653/1000000 [11:20<4:13:16, 63.59step/s, ext_r=0.00, wm=51.15, epD=46250.85, intrD=0.11]Episode 165

以下で利用可能なAtari環境を確認できます。

In [ ]:
import gymnasium as gym
import ale_py
gym.register_envs(ale_py)

# List available Atari v5 environments
atari_envs = [env_id for env_id in gym.registry.keys() if "ALE/" in env_id and "-v5" in env_id]
atari_envs.sort()

print(f"Available Atari Envs: {len(atari_envs)}")
print("All Environments:")
for env in atari_envs:
    print(env)

Available Atari Envs: 104
All Environments:
ALE/Adventure-v5
ALE/AirRaid-v5
ALE/Alien-v5
ALE/Amidar-v5
ALE/Assault-v5
ALE/Asterix-v5
ALE/Asteroids-v5
ALE/Atlantis-v5
ALE/Atlantis2-v5
ALE/Backgammon-v5
ALE/BankHeist-v5
ALE/BasicMath-v5
ALE/BattleZone-v5
ALE/BeamRider-v5
ALE/Berzerk-v5
ALE/Blackjack-v5
ALE/Bowling-v5
ALE/Boxing-v5
ALE/Breakout-v5
ALE/Carnival-v5
ALE/Casino-v5
ALE/Centipede-v5
ALE/ChopperCommand-v5
ALE/CrazyClimber-v5
ALE/Crossbow-v5
ALE/Darkchambers-v5
ALE/Defender-v5
ALE/DemonAttack-v5
ALE/DonkeyKong-v5
ALE/DoubleDunk-v5
ALE/Earthworld-v5
ALE/ElevatorAction-v5
ALE/Enduro-v5
ALE/Entombed-v5
ALE/Et-v5
ALE/FishingDerby-v5
ALE/FlagCapture-v5
ALE/Freeway-v5
ALE/Frogger-v5
ALE/Frostbite-v5
ALE/Galaxian-v5
ALE/Gopher-v5
ALE/Gravitar-v5
ALE/Hangman-v5
ALE/HauntedHouse-v5
ALE/Hero-v5
ALE/HumanCannonball-v5
ALE/IceHockey-v5
ALE/Jamesbond-v5
ALE/JourneyEscape-v5
ALE/Kaboom-v5
ALE/Kangaroo-v5
ALE/KeystoneKapers-v5
ALE/KingKong-v5
ALE/Klax-v5
ALE/Koolaid-v5
ALE/Krull-v5
ALE/KungFuMa